### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

### Start with our Message class

In [2]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [3]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

### Let's reintroduce a tool

In [4]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [5]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [6]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OllamaChatCompletionClient(model="qwen3:8b")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OllamaChatCompletionClient(model="qwen3:8b")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OllamaChatCompletionClient(model="qwen3:8b")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [7]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [8]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [9]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are the key pros of using AutoGen for AI agent projects:  

1. **Multi-Agent Collaboration**: Enables seamless communication and task coordination between multiple AI agents, fostering teamwork and complex problem-solving.  
2. **Flexibility & Customization**: Allows agents to be defined with specific roles, capabilities, and communication protocols, tailored to project needs.  
3. **Integration Capabilities**: Supports integration with APIs, tools, external systems, and human input, enhancing functionality and adaptability.  
4. **Scalability & Efficiency**: Designed for building scalable, efficient AI applications with clear tool orchestration and memory management.  
5. **Compatibility**: Works with diverse LLMs, toolchains, and frameworks, ensuring versatility across different AI ecosystems.  
6. **Open-Source & Community-Driven**: Open-source framework with active development, offering transparency and community support for customization.  

These features make AutoGen a robust choice for creating collaborative, scalable, and adaptable AI agent systems. TERMINATE

## Cons of AutoGen:
Here are key reasons against choosing AutoGen for an AI agent project, based on the research:

1. **Steep Learning Curve & Poor Documentation**:  
   AutoGen’s documentation is often criticized for being unclear and lacking sufficient examples, making it difficult for developers to onboard quickly or troubleshoot issues.

2. **Over-Engineering Risk**:  
   AutoGen’s focus on multi-agent systems and complex workflows may lead to over-engineered solutions for projects that require simpler, single-agent implementations. This can increase complexity without proportional benefits.

3. **High Cost of Multi-Agent Workflows**:  
   Running multiple agents (e.g., for coordination or task splitting) is resource-intensive and may not be cost-effective for many real-world use cases, especially smaller projects or those with tight budgets.

4. **Intensive Prompt Engineering**:  
   AutoGen requires detailed, algorithmic prompts to guide agent interactions, which can be time-consuming to design and debug. This increases development time and expertise requirements.

5. **Limited No-Code/Low-Code Support**:  
   Unlike platforms like CrewAI, AutoGen lacks visual tools or rapid prototyping features, making it less accessible for teams without strong coding skills or for projects needing quick iterations.

6. **Niche Use Case Fit**:  
   AutoGen excels in highly complex, computation-heavy tasks (e.g., multi-agent collaboration), but may not be ideal for structured workflows or simpler automation tasks better handled by alternatives like CrewAI.

If your project prioritizes simplicity, cost-efficiency, or rapid development, AutoGen’s drawbacks may outweigh its benefits. Consider alternatives if your needs align with these limitations. TERMINATE.



## Decision:

**Decision:** Use AutoGen if the project requires complex multi-agent collaboration, scalability, and integration with diverse tools/LLMs. Avoid AutoGen for simpler workflows, cost-sensitive applications, or teams needing rapid, low-code development.  

**Rationale:** AutoGen’s strengths in multi-agent systems, flexibility, and open-source community make it ideal for advanced, collaborative AI projects. However, its steep learning curve, over-engineering risks, and resource intensity may hinder adoption for simpler or budget-constrained tasks. The choice hinges on whether the project’s complexity and long-term scalability justify overcoming these challenges. TERMINATE

In [10]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [11]:
await host.stop()